In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('illiteracy.txt', sep='\t')

In [8]:
data.iloc[:, [1, 2]].corr(method='pearson')

,Illit,Births
Illit,1.000000,0.768663
Births,0.768663,1.000000


In [10]:
data['Illit'].corr(data['Births'])

0.7686630271151789

In [9]:
data.iloc[:, [1, 2]].corr(method='spearman')

,Illit,Births
Illit,1.000000,0.752962
Births,0.752962,1.000000


In [11]:
water = pd.read_csv('water.txt', sep='\t')

In [13]:
water['mortality'].corr(water['hardness'])

-0.6548486232042463

In [16]:
water['mortality'].corr(water['hardness'], method='spearman')

-0.6316646189166502

In [17]:
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [20]:
water.loc[water['location'] == 'South', ['mortality', 'hardness']].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [21]:
water.loc[water['location'] == 'North', ['mortality', 'hardness']].corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


In [22]:
def mcc(a, b, c, d):
    denominator = np.sqrt((a+b)*(a+c)*(b+d)*(c+d))
    return (a*d - b*c) / denominator

In [23]:
mcc(203, 718, 239, 515)

-0.10900237458678963

In [24]:
from scipy import stats

In [31]:
stats.chi2_contingency(np.array([[203, 718], [239, 515]]))

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 677.96597015],
        [198.96597015, 555.03402985]]))

In [32]:
def proportions_diff_confint_ind(sample1, sample2, alpha=0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [33]:
women = np.append(np.ones(203), np.zeros(718))

In [35]:
men = np.append(np.ones(239), np.zeros(515))

In [36]:
proportions_diff_confint_ind(men, women)

(0.053905233215813156, 0.13922183141523897)

In [37]:
def z_stat_ind(sample1, sample2):
    p1 = sum(sample1) / len(sample1)
    p2 = sum(sample2) / len(sample2)
    n1, n2 = len(sample1), len(sample2)
    P = (p1 * n1 + p2 * n2) / (n1 + n2)
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1 / n1 + 1 / n2))

In [40]:
z = z_stat_ind(men, women)

In [41]:
2 * (1 - stats.norm.cdf(z))

8.153453089576601e-06

In [50]:
survey = pd.DataFrame(columns=[0, 1, 2], index=[0, 1, 2])

In [51]:
survey.iloc[0, :] = [197, 111, 33]

In [52]:
survey.iloc[1, :] = [382, 685, 331]
survey.iloc[2, :] = [110, 342, 333]

In [53]:
survey

,0,1,2
0,197,111,33
1,382,685,331
2,110,342,333


In [64]:
res = 0
n = 0
for i in range(survey.shape[0]):
    n_i_plus = sum(survey.iloc[i, :])
    n += n_i_plus
    for j in range(survey.shape[1]):
        n_j_plus = sum(survey.iloc[:, j])
        n += n_j_plus
        res += (survey.iloc[i, j] ** 2) / (n_i_plus + n_j_plus)
res = n_ * (res - 1)

In [65]:
res

1254825.4499714614

In [62]:
n_ = 0
for i in range(3):
    n_ += sum(survey.loc[i]) 

In [63]:
n_

2524

In [67]:
chi = stats.chi2_contingency(survey)[0]

In [69]:
chi

293.68311039689746

In [72]:
np.sqrt(chi / (n_ * 2))

0.2412013934500338

## Множественные гипотезы

In [73]:
aucs = pd.read_csv('AUCs.txt' ,sep='\t')

In [76]:
aucs.columns = ['name', 'C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']

In [77]:
aucs

,name,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [104]:
from statsmodels.stats.descriptivestats import sign_test
from scipy import stats

In [79]:
import itertools

In [105]:
sign_tests = {}
for idx in itertools.permutations(range(1, 5), 2):
    sign_tests[idx] = stats.wilcoxon(aucs.iloc[:, idx[0]], aucs.iloc[:, idx[1]])[1]

In [106]:
sign_tests

{(1, 2): 0.01075713311978963,
 (1, 3): 0.861262330095348,
 (1, 4): 0.015906444101703374,
 (2, 1): 0.01075713311978963,
 (2, 3): 0.046332729793395394,
 (2, 4): 0.3278256758446406,
 (3, 1): 0.861262330095348,
 (3, 2): 0.046332729793395394,
 (3, 4): 0.022909099354356588,
 (4, 1): 0.015906444101703374,
 (4, 2): 0.3278256758446406,
 (4, 3): 0.022909099354356588}

In [107]:
sorted(sign_tests.items(), key=lambda item : item[1])

[((1, 2), 0.01075713311978963),
 ((2, 1), 0.01075713311978963),
 ((1, 4), 0.015906444101703374),
 ((4, 1), 0.015906444101703374),
 ((3, 4), 0.022909099354356588),
 ((4, 3), 0.022909099354356588),
 ((2, 3), 0.046332729793395394),
 ((3, 2), 0.046332729793395394),
 ((2, 4), 0.3278256758446406),
 ((4, 2), 0.3278256758446406),
 ((1, 3), 0.861262330095348),
 ((3, 1), 0.861262330095348)]

In [108]:
perm = []
for i in range(1, 5):
    for j in range(i + 1, 5):
        perm.append((i, j))

In [109]:
signs = pd.Series(index=perm)

In [110]:
for i in range(1, 5):
    for j in range(i + 1, 5):
        signs[(i, j)] = stats.wilcoxon(aucs.iloc[:, i], aucs.iloc[:, j])[1]

In [111]:
signs.values

array([0.01075713, 0.86126233, 0.01590644, 0.04633273, 0.32782568,
       0.0229091 ])

In [114]:
signs

(1, 2)    0.010757
(1, 3)    0.861262
(1, 4)    0.015906
(2, 3)    0.046333
(2, 4)    0.327826
(3, 4)    0.022909
dtype: float64

In [90]:
from statsmodels.stats.multitest import multipletests

In [115]:
multipletests(signs.values, method='holm')

(array([False, False, False, False, False, False]),
 array([0.0645428 , 0.86126233, 0.07953222, 0.13899819, 0.65565135,
        0.0916364 ]),
 0.008512444610847103,
 0.008333333333333333)

In [116]:
multipletests(signs.values, method='fdr_bh')

(array([ True, False,  True, False, False,  True]),
 array([0.0458182 , 0.86126233, 0.0458182 , 0.06949909, 0.39339081,
        0.0458182 ]),
 0.008512444610847103,
 0.008333333333333333)

## Тест "Практика построения регрессии"

In [2]:
data = pd.read_csv('botswana.tsv', sep='\t')

In [3]:
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


In [4]:
data['religion'].nunique()

4

In [5]:
sum(np.sum(data.isna(), axis=1) == 0)

1834

In [6]:
data['nevermarr'] = 1 - data['evermarr']

In [7]:
data.drop('evermarr', axis=1, inplace=True)

In [8]:
data['agefm'].fillna(0, inplace=True)

In [9]:
data.loc[data['nevermarr'] == 1, 'heduc'] = -1

In [10]:
sum(data['heduc'].isna())

123

In [11]:
sum(data['idlnchld'] == np.NaN)

0

In [12]:
data['idlnchld_noans'] = (data['idlnchld'].isna()).astype(int)

In [13]:
data.loc[data['idlnchld_noans'] == 1, 'idlnchld'] = -1

In [14]:
def fill_noans(df, column_name, value):
    df[f'{column_name}_noans'] = (df[column_name].isna()).astype(int)
    df.loc[data[f'{column_name}_noans'] == 1, column_name] = value
    return df

In [15]:
data = fill_noans(data, 'heduc', -2)

In [16]:
data = fill_noans(data, 'usemeth', -1)

In [17]:
nans = np.sum(data.isna(), axis=0)
names = list(nans[nans > 0].index)

In [18]:
n = np.sum(data[names].isna(), axis=1) 
index_to_drop = list(n[n > 0].index)

In [19]:
data.drop(index_to_drop, inplace=True)

In [20]:
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1,0,0,0
4357,2,28,7,protestant,4.0,1.0,1.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1,0,0,0
4358,4,24,5,protestant,4.0,1.0,1.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1,0,0,0
4359,1,26,0,spirit,5.0,1.0,0.0,22.0,7.0,0,0.0,1.0,0.0,0.0,0,0,0,0


In [21]:
data.shape[0] * data.shape[1]

78264

In [22]:
import statsmodels.formula.api as smf
import patsy

In [118]:
!pip install -U patsy

  Using cached https://files.pythonhosted.org/packages/ea/0c/5f61f1a3d4385d6bf83b83ea495068857ff8dfb89e74824c6e9eb63286d8/patsy-0.5.1-py2.py3-none-any.whl
  Found existing installation: patsy 0.5.0
    Uninstalling patsy-0.5.0:
      Successfully uninstalled patsy-0.5.0
You should consider upgrading via the 'pip install --upgrade pip' command.


In [126]:
!pip install -U statsmodels

  Using cached https://files.pythonhosted.org/packages/f0/68/7e7ce7a1c6f4ec65083ae9be678d55a3c32d2b12baa579824fffba9b082e/statsmodels-0.11.1-cp36-cp36m-macosx_10_13_x86_64.whl
  Found existing installation: statsmodels 0.9.0
    Uninstalling statsmodels-0.9.0:
      Successfully uninstalled statsmodels-0.9.0
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
query = 'ceb ~ ' + ' + '.join(data.columns[1:])

In [34]:
m1 = smf.ols(query, data=data)
fitted = m1.fit(cov_type='HC1')

In [35]:
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Mon, 17 Aug 2020   Prob (F-statistic):               0.00
Time:                        00:31:33   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [28]:
data['religion'].value_counts()

spirit        1838
other         1076
protestant     989
catholic       445
Name: religion, dtype: int64

In [31]:
import statsmodels.stats.api as sms

In [36]:
sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1]

1.1452927633445016e-225

In [37]:
feat_to_delete = ['religion', 'tv', 'radio']
data_del = data.drop(feat_to_delete, axis=1)

In [40]:
query_del = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

In [43]:
m2 = smf.ols(query_del, data=data_del)
fitted2 = m2.fit(cov_type='HC1')

In [45]:
sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)

(1112.4700385717538,
 1.1197458896536614e-228,
 106.41517187062938,
 3.592611312536214e-265)

In [47]:
fitted.compare_f_test(fitted2)

/Users/amsavchenko/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:2135: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(0.9192357784628133, 0.46723055472768693, 5.0)

In [49]:
data_del_meth = data_del.drop(['usemeth_noans', 'usemeth'], axis=1)

In [50]:
query3 = 'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans'

In [52]:
fitted3 = smf.ols(query3, data=data_del_meth).fit(cov_type='HC1')

In [53]:
fitted2.compare_f_test(fitted3)

/Users/amsavchenko/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:2135: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(92.89058230109666, 3.155200948041877e-40, 2.0)

In [54]:
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 17 Aug 2020   Prob (F-statistic):               0.00
Time:                        00:40:48   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.